In [0]:
%run "/Repos/017shashank.k.mallick@gmail.com/AzureDatabricks_Self/Workspace_DEV/TestingNotebook"

# DELTA LAKE

In [0]:
cols = [i.strip().replace(" ","_").split(',')[0] for i in df_happiness.columns]
df_happiness = df_happiness.toDF(*cols)
df_happiness.write.format("delta")\
    .mode("append")\
        .option('path','abfss://destination@storagefordatafactory017.dfs.core.windows.net/deltaTables')\
            .save()

## Managed vs External Delta Tables

### Database

In [0]:
%sql
CREATE DATABASE mysqlDB;

### Managed Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS mysqlDB.happiness
(
  id INT,
  name STRING,
  age INT
)
USING DELTA;

In [0]:
%sql
INSERT INTO mysqlDB.happiness
VALUES
(1, 'John', 25),
(2, 'Jane', 30),
(3, 'Bob', 40),
(4, 'Alice', 50),
(5, 'Charlie', 60);


In [0]:
%sql
select * from mysqldb.happiness;

In [0]:
%sql
DROP TABLE mysqldb.happiness;

### External Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS mysqlDB.happiness_ext
(
  id INT,
  name STRING,
  age INT
)
USING DELTA 
LOCATION 'abfss://destination@storagefordatafactory017.dfs.core.windows.net/mysqlDB/externalTables';

In [0]:
%sql
INSERT INTO mysqlDB.happiness_ext
VALUES
(1, 'John', 25),
(2, 'Jane', 30),
(3, 'Bob', 40),
(4, 'Alice', 50),
(5, 'Charlie', 60);


In [0]:
%sql
SELECT * FROM mysqldb.happiness;

In [0]:
%sql
DROP TABLE mysqldb.happiness;

# Delta Tables Functionalities

### INSERT

In [0]:
%sql
INSERT INTO mysqlDB.happiness_ext
VALUES
(6, 'Shashank', 25),
(7, 'Rishabh', 30);

In [0]:
%sql
select * from mysqldb.happiness_ext;

### DELETE

In [0]:
%sql
DELETE FROM mysqldb.happiness_ext
where id = 7;

In [0]:
%sql
SELECT * FROM mysqldb.happiness_ext;

### DATA VERSIONING

In [0]:
%sql
DESCRIBE HISTORY mysqldb.happiness_ext;

### TIME TRAVEL

In [0]:
%sql
RESTORE TABLE mysqldb.happiness_ext VERSION AS OF 2;


In [0]:
%sql
select * from mysqldb.happiness_ext;

### VACUUM

In [0]:
%sql
VACUUM mysqldb.happiness_ext RETAIN 0 HOURS;

# DELTA TABLE OPTIMIZATION

### OPTIMIZE

In [0]:
%sql
select * from mysqldb.happiness_ext;

In [0]:
%sql
OPTIMIZE mysqldb.happiness_ext;

In [0]:
%sql
select * from mysqldb.happiness_ext;

### ZORDER BY

In [0]:
%sql
OPTIMIZE mysqldb.happiness_ext ZORDER BY (id);

In [0]:
%sql
select * from mysqldb.happiness_ext;

# Deep Clone vs Shallow Clone

### DEEP CLONE

In [0]:
%sql
CREATE TABLE mysqldb.deep_clone_happiness 
DEEP CLONE mysqldb.happiness_ext;

In [0]:
%sql
INSERT INTO mysqldb.happiness_ext
VALUES
(8,'Mr Shashank', 27);

In [0]:
%sql
SELECT * from mysqldb.happiness_ext;

In [0]:
%sql
SELECT * from mysqldb.deep_clone_happiness;

In [0]:
%sql
describe extended mysqldb.deep_clone_happiness;

### SHALLOW CLONE

In [0]:
%sql
CREATE TABLE IF NOT EXISTS mysqldb.happiness_int
AS
SELECT * FROM mysqldb.happiness_ext;

In [0]:
%sql
describe formatted mysqldb.happiness_ext;

In [0]:
#create internal table to create a shallow copy